In [1]:
!pip install confluent-kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 35.1 MB/s eta 0:00:00


In [8]:
pip install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.4 MB/s eta 0:00:00


In [12]:
from confluent_kafka import Producer, Consumer, KafkaError
from faker import Faker
import json

In [13]:
fake = Faker('en_US')

In [11]:
# Create a Kafka producer instance
producer_config = {
    'bootstrap.servers': 'pkc-lzvrd.us-west4.gcp.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'Z2W5M7OMMQG3IH6I',
    'sasl.password': 'Y68+Y6NHrTpMzq/ETj9tj5m1K7kScjdA1dc1aO6L516S9qGTqGX09gNXk/Ol7c+Q',  
}

In [6]:
# Create a Kafka cosumer instance
consumer_conf = {
    'bootstrap.servers': 'pkc-lzvrd.us-west4.gcp.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'Z2W5M7OMMQG3IH6I',
    'sasl.password': 'Y68+Y6NHrTpMzq/ETj9tj5m1K7kScjdA1dc1aO6L516S9qGTqGX09gNXk/Ol7c+Q',
    'group.id': 'cluster_0',
    'auto.offset.reset': 'earliest'
}

consumer = Consumer(consumer_conf)

In [21]:
topic = 'sample_data'

In [25]:
#Create a function for indegisting Data into a Kafka Cluster

def Produce():

  #Define number of messages to generate
    num_messages = 20
          
    # Initialize Kafka producer
    producer = Producer(producer_config)


    #Create Synthetic data to be ingested into Kafka Cluster
    for i in range(num_messages):
        data = {
            'transaction_id':fake.random_int(),
            'sender_phone_number': fake.phone_number(),
            'receiver_phone_number': fake.phone_number(),
            'transaction_amount': fake.random_int(),
            'transaction_time"':fake.time()
        }
        
        # Serialize data to JSON and send to Kafka topic
        producer.produce(topic, key=str(i), value=json.dumps(data))
        producer.flush()

In [ ]:
#Create a function for Extracting Data from Kafka Cluster

def Extract():

# Create Kafka consumer and subscribe to input topic

   consumer = Consumer(consumer_conf)
   consumer.subscribe([topic])

#Define an empty list to store the JSON data
json_data_list = []


#Define max messages and message count variables so as to create a condition to exit the While loop
max_messages = 20
message_count = 0

# Consume data from Kafka
while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print('Reached end of partition')
            else:
                print('Error while consuming message: {}'.format(msg.error()))
        else:
            # Convert the message value from bytes to string
            json_string = msg.value().decode('utf-8')
            
            # Parse the JSON data
            json_data = json.loads(json_string)
            
            # Append the JSON data to the list
            json_data_list.append(json_data)
            
            #Increment the message counter
            message_count += 1

            # Exit the loop when we have consumed the maximum number of messages set in max_messages variable
            if message_count >= max_messages:
                break